In [1]:
import sys
import time
import numpy as np
from pathlib import Path
from tqdm import tqdm
from typing import Callable, Dict, List

# pytket imports
from pytket import Circuit, OpType
from pytket.passes import DecomposeBoxes, AutoRebase, FullPeepholeOptimise

# add path to your local tket2_rewriting repo
sys.path.append("../../../tket2-rewriting/tket2_rewriting")

# import your custom optimisers
from cx_gadget.cx_gadget import CXGadget
from zzphase_flip.zzphase_flip import ZZPhaseFlip
from three_qubit_squash.three_qubit_squash_gadget import Squash3Gadget
from label.label_gadget import LabelGadget
from KAK.kak_gadget import KAKGadget
from tket.circuit import Tk2Circuit
from tket.optimiser import BadgerOptimiser

from circuit_generation.cx_gadget_random import CXGadgetBenchmarkCircuit
from circuit_generation.cx_gadget_vanilla import CXGadgetVanillaBenchmarkCircuit
from circuit_generation.cx_many_rz import CXManyRzBenchmarkCircuit
from circuit_generation.gadget import (
    SquashGadgetTestCircuit,
    LabelFriendlyCircuit,
    SimpleLabelFriendlyCircuit,
    KAKFriendlyCircuit,
)

In [2]:
N_SAMPLES = 10
WIDTH = 8
DEPTH = 10
RNG = np.random.default_rng(42)

# List of circuit generators to benchmark
GENERATORS: Dict[str, Callable[[], Circuit]] = {
    "CXGadgetBenchmarkCircuit": lambda: CXGadgetBenchmarkCircuit(WIDTH, DEPTH, RNG),
    "CXGadgetVanillaBenchmarkCircuit": lambda: CXGadgetVanillaBenchmarkCircuit(WIDTH, DEPTH, RNG),
    "CXManyRzBenchmarkCircuit": lambda: CXManyRzBenchmarkCircuit(WIDTH, DEPTH, RNG),
    "SquashGadgetTestCircuit": SquashGadgetTestCircuit,
    "LabelFriendlyCircuit": lambda: LabelFriendlyCircuit(WIDTH, DEPTH, RNG),
    "SimpleLabelFriendlyCircuit": lambda: SimpleLabelFriendlyCircuit(WIDTH, DEPTH, RNG),
    "KAKFriendlyCircuit": lambda: KAKFriendlyCircuit(WIDTH, DEPTH, RNG),
}

In [3]:
GADGET_INSTANCES = [
    ZZPhaseFlip(),
    CXGadget(),
    Squash3Gadget(),
    # LabelGadget(),
    KAKGadget(),
]
GADGET_NAMES = ["ZZPhaseFlip", "CXGadget", "Squash3Gadget", "KAKGadget"]

OPTIMISER = BadgerOptimiser(GADGET_INSTANCES)

In [4]:
def _try_apply_gadget(gadget, tk2_circ):
    """
    Attempt to apply a gadget to a Tk2Circuit using a sequence of likely method names.
    If the gadget returns a new circuit, that is used. If it modifies in-place, the
    original object is used. Raises AttributeError if no known method exists.
    """
    # Candidate method names (ordered by likelihood)
    try_methods = ("apply", "rewrite", "run", "rewrite_tk2", "rewrite_circuit", "transform")

    for m in try_methods:
        if hasattr(gadget, m):
            func = getattr(gadget, m)
            res = func(tk2_circ)
            # if the method returns something (new circuit), use it, otherwise assume in-place
            return tk2_circ if res is None else res

    # Final fallback: if gadget is callable, call it
    if callable(gadget):
        res = gadget(tk2_circ)
        return tk2_circ if res is None else res

    raise AttributeError(f"No known apply/rewrite/run interface for gadget {gadget!r}")


In [5]:
def benchmark_generator(name: str, generator: Callable[[], Circuit], n_samples: int = N_SAMPLES):
    """
    For each generated circuit:
      - Run DecomposeBoxes + AutoRebase
      - Measure original 2Q gates
      - (A) Run OPTIMISER.optimise(...) (timed) -> measure opt_2q
      - (B) On a separate copy, apply gadgets sequentially using _try_apply_gadget to
            collect per-gadget times and 2Q reductions
      - Apply FullPeepholeOptimise() to the optimiser result and measure final 2Q
    Returns aggregated metrics across n_samples.
    """
    total_opt_time = 0.0
    total_peephole_time = 0.0

    orig_2q_total = 0
    opt_2q_total = 0
    peephole_2q_total = 0

    # accumulate gadget metrics (over all samples)
    gadget_time_totals = {n: 0.0 for n in GADGET_NAMES}
    gadget_2q_reduction = {n: 0 for n in GADGET_NAMES}
    gadget_failures = {n: 0 for n in GADGET_NAMES}

    for _ in tqdm(range(n_samples), desc=f"Benchmarking {name}"):
        # generate and preprocess
        circ = generator()
        DecomposeBoxes().apply(circ)
        AutoRebase({OpType.CX, OpType.Rz, OpType.Rx}).apply(circ)

        # record original 2Q count
        orig_2q = circ.n_2qb_gates()
        orig_2q_total += orig_2q

        # -------------------------
        # (A) Run whole-optimiser (works for you)
        # -------------------------
        tk2_for_opt = Tk2Circuit(circ)  # fresh instance for optimiser
        t0 = time.time()
        opt_circ_tk2 = OPTIMISER.optimise(tk2_for_opt)
        t_opt = time.time() - t0
        total_opt_time += t_opt

        # convert to pytket Circuit and measure 2Q gates after optimiser
        opt_circ1 = opt_circ_tk2.to_tket1()
        opt_2q = opt_circ1.n_2qb_gates()
        opt_2q_total += opt_2q

        # -------------------------
        # (B) Per-gadget sequential profiling on a copy of the original circuit
        #     (apply gadgets one-by-one in the same order as GADGET_INSTANCES)
        # -------------------------
        tk2_copy = Tk2Circuit(circ)  # fresh copy to apply gadgets sequentially
        prev_count = orig_2q

        for gname, gadget in zip(GADGET_NAMES, GADGET_INSTANCES):
            try:
                gstart = time.time()
                tk2_copy = _try_apply_gadget(gadget, tk2_copy)
                gtime = time.time() - gstart
                gadget_time_totals[gname] += gtime

                # measure 2Q after this gadget (convert to tket1)
                cur_2q = tk2_copy.to_tket1().n_2qb_gates()
                delta = max(prev_count - cur_2q, 0)
                gadget_2q_reduction[gname] += delta
                prev_count = cur_2q

            except Exception as exc:
                # record failure but continue
                gadget_failures[gname] += 1
                # do not change prev_count or tk2_copy in this case
                continue

        # -------------------------
        # (C) Apply Full Peephole to optimiser result and measure
        # -------------------------
        peephole_pass = FullPeepholeOptimise()
        t0 = time.time()
        peephole_pass.apply(opt_circ1)
        t_peep = time.time() - t0
        total_peephole_time += t_peep

        peephole_2q = opt_circ1.n_2qb_gates()
        peephole_2q_total += peephole_2q

    reduction_total = orig_2q_total - opt_2q_total
    pct_reduction = 100.0 * reduction_total / orig_2q_total if orig_2q_total > 0 else 0.0
    avg_opt_time = total_opt_time / n_samples
    avg_peephole_time = total_peephole_time / n_samples

    # gadget time percentages (normalised over total gadget time collected)
    total_gadget_time_collected = sum(gadget_time_totals.values())
    gadget_percentages = {
        g: (100.0 * t / total_gadget_time_collected) if total_gadget_time_collected > 0 else 0.0
        for g, t in gadget_time_totals.items()
    }

    # gadget contribution to 2Q reduction (percentage of original total 2Q gates)
    gadget_2q_pct = {
        g: (100.0 * red / orig_2q_total) if orig_2q_total > 0 else 0.0
        for g, red in gadget_2q_reduction.items()
    }

    return {
        "name": name,
        "orig_2q": orig_2q_total,
        "opt_2q": opt_2q_total,
        "peephole_2q": peephole_2q_total,
        "reduction": reduction_total,
        "pct_reduction": pct_reduction,
        "total_opt_time": total_opt_time,
        "avg_opt_time": avg_opt_time,
        "total_peephole_time": total_peephole_time,
        "avg_peephole_time": avg_peephole_time,
        "gadget_times": gadget_time_totals,
        "gadget_percentages": gadget_percentages,
        "gadget_2q_reduction": gadget_2q_reduction,
        "gadget_2q_pct": gadget_2q_pct,
        "gadget_failures": gadget_failures,
    }

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

results = []

print(f"\nRunning benchmarking suite on {len(GENERATORS)} circuit types (with gadget profiling)...")

with ThreadPoolExecutor() as executor:
    # Submit all tasks
    futures = {executor.submit(benchmark_generator, name, gen, N_SAMPLES): name for name, gen in GENERATORS.items()}

    # Print results as they complete
    for future in as_completed(futures):
        res = future.result()
        results.append(res)  # optional, if you want to store all results
        print(
            f"{res['name']:<35s} "
            f"2Q Gates: {res['orig_2q']:>6d} → {res['opt_2q']:>6d} "
            f"({res['pct_reduction']:.2f}% gadget reduction) → {res['peephole_2q']:>6d} after peephole | "
            f"Avg Optimiser Time: {res['avg_opt_time']*1000:.2f} ms | Avg Peephole: {res['avg_peephole_time']*1000:.2f} ms"
        )

    # print("  Gadget time distribution (percent of collected gadget time):") #BROKEN
    # for g in GADGET_NAMES:
    #     print(f"    {g:<15s}: {r['gadget_percentages'][g]:6.2f}%  | 2Q reduction: {r['gadget_2q_reduction'][g]:>6d} ({r['gadget_2q_pct'][g]:5.2f}%)  failures: {r['gadget_failures'][g]}")
    # print()



Running benchmarking suite on 7 circuit types (with gadget profiling)...



Benchmarking CXGadgetBenchmarkCircuit:   0%|          | 0/10 [00:00<?, ?it/s]AttributeError: SingleQubit














Benchmarking SquashGadgetTestCircuit: 100%|██████████| 10/10 [00:00<00:00, 79.95it/s]


SquashGadgetTestCircuit             2Q Gates:      0 →      0 (0.00% gadget reduction) →      0 after peephole | Avg Optimiser Time: 0.40 ms | Avg Peephole: 2.99 ms


















































Benchmarking KAKFriendlyCircuit: 100%|██████████| 10/10 [00:10<00:00,  1.08s/it]


KAKFriendlyCircuit                  2Q Gates:    169 →    159 (5.92% gadget reduction) →    121 after peephole | Avg Optimiser Time: 876.91 ms | Avg Peephole: 196.24 ms


bit
Benchmarking CXManyRzBenchmarkCircuit:  10%|█         | 1/10 [00:11<01:40, 11.16s/it]ot match op Rz: AttributeError: SingleQubit
Wa

In [ ]:
# Total runtime across all benchmarks
total_time = sum(r["total_opt_time"] + r["total_peephole_time"] for r in results)
print(f"\nTotal runtime for all benchmarks (optimiser + peephole): {total_time:.2f} s")

# Total 2-qubit gate reduction across all benchmarks
total_orig_2q = sum(r["orig_2q"] for r in results)
total_final_2q = sum(r["peephole_2q"] for r in results)
total_reduction = total_orig_2q - total_final_2q
total_pct_reduction = 100.0 * total_reduction / total_orig_2q if total_orig_2q > 0 else 0.0

print(f"Total 2Q gates: {total_orig_2q} → {total_final_2q} "
      f"({total_reduction} reduced, {total_pct_reduction:.2f}% reduction)")


Total runtime for all benchmarks (optimiser + peephole): 19.28 s
Total 2Q gates: 68 → 38 (30 reduced, 44.12% reduction)
